## Importing packages

In [2]:
try:
    import selenium
    from selenium import webdriver
except:
    !pip install selenium
    import selenium
    from selenium import webdriver

import ast

try:
    import pyautogui
except:
    !pip install pyautogui
    import pyautogui
    
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup

import pandas as pd
import time
import random

import os

# from googlesearch import search

from pandas.io.html import read_html

## Defining path - chrome web driver

In [14]:
# Download the chromedriver that aligns with your chrome browser version.
# This chromedriver is for version 84

driver_path = str(os.getcwd()) + "/Utilities/chromedriver"

In [15]:
# Test the chromedriver, if a new window opens the chromedriver is working
open_browser(driver_path)

<selenium.webdriver.chrome.webdriver.WebDriver (session="eaf7f051597b696c942986d23ea9aacb")>

## Create re-usable functions

In [16]:
##########################################################
# Function: open chrome driver
def open_browser(chromedriver_path):
    driver = webdriver.Chrome(chromedriver_path)
    return driver

##########################################################
# Function: Get desired element from webpage
# Required structure of dict
# input_dict = {'search_by': 'by_class','find_type': 'single', 'find_str': 'table-responsive'}    
def find_element(driver, input_dict):
    try:
        
        search_by = input_dict['search_by']

        find_type = input_dict['find_type']
        
        if find_type == 'single':
            if search_by == "by_id":
                item = driver.find_element_by_id(input_dict['find_str'])
                
            elif search_by == "by_name":
                item = driver.find_element_by_name(input_dict['find_str'])
                
            elif search_by == "by_xpath":
                item = driver.find_element_by_xpath(input_dict['find_str'])
                
            elif search_by == "by_link_text":
                item = driver.find_element_by_link_text(input_dict['find_str'])
                
            elif search_by == "by_partial_link_text":
                item = driver.find_element_by_partial_link_text(input_dict['find_str'])
                
            elif search_by == "by_tag_name":
                item = driver.find_element_by_tag_name(input_dict['find_str'])
                
            elif search_by == "by_class":
                item = driver.find_element_by_class_name(input_dict['find_str'])
                
            elif search_by == "by_css_selector":
                item = driver.find_element_by_css_selector(input_dict['find_str'])

        elif find_type == "multiple":
            if search_by == "by_name":
                item = driver.find_elements_by_name(input_dict['find_str'])
                
            elif search_by == "by_xpath":
                item = driver.find_elements_by_xpath(input_dict['find_str'])
                
            elif search_by == "by_link_text":
                item = driver.find_elements_by_link_text(input_dict['find_str'])
                
            elif search_by == "by_partial_link_text":
                item = driver.find_elements_by_partial_link_text(input_dict['find_str'])
                
            elif search_by == "by_tag_name":
                item = driver.find_elements_by_tag_name(input_dict['find_str'])
                
            elif search_by == "by_class":
                item = driver.find_elements_by_class_name(input_dict['find_str'])
                
            elif search_by == "by_css_selector":
                item = driver.find_elements_by_css_selector(input_dict['find_str'])
        
        return item
    
    except Exception as e:
        print(e)
        
##########################################################      
# Get the html code of certain webpage with input as url
def get_html_webpage(url):
    try:
        driver = open_browser(driver_path)
        driver.get(url)
        html_source = driver.page_source
        return html_source
    except Exception as e:
        print(e)

##########################################################
# Get all list items from Wikipedia or any url
def get_list_items_from_wiki(url):
    output = []
    
    # Getting html code of web page
    html_source = get_html_webpage(url)
    
    # Using soup for easier traversing 
    soup = BeautifulSoup(html_source, "html.parser")

    for tag in soup.find_all("li"):  
        # Getting the link text of list item
        tag_name = tag.text
        output.append(tag_name)
    return output
    
##########################################################
# To extract all government agencies of country and it's description
def get_gov_agency(Country):
    
    # Open browser
    driver = open_browser(driver_path)
    driver.get('https://www.google.com/')

    # Write search term in search bar
    pyautogui.write('Government agencies of ' + Country)
    
    # Introducing randomly 1 or 2 second lag to prevent google from blocking the bot
    time.sleep(random.randint(1,2))
    
    # Using pyautogui to execute Enter button in keyboard
    pyautogui.press('enter')
    
    try:
        # Getting the cards of gov agencies
        input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': "extabar"}

        item = find_element(driver, input_dict)
        item = item.get_attribute('innerHTML')

        soup = BeautifulSoup(item, "html.parser")
    
        output_dict = {}
        
        # Iterating through all cards
        for tag in soup.find_all("a"):
            
            # Introducing randomly 1 or 2 second lag to prevent google from blocking the bot
            time.sleep(random.randint(1,2))
            
            # Escaping two non desirable links in the cards
            if tag.text == "Government Agencies" or tag.text == "Government" or tag.text == Country:
                continue
            else:
                # Using the card name to create search_term and get the element
                
                temp = tag.text
                
                # if-else condition because the name might contain ' or " in the name, so to avoid technical error
                if '"' in temp:
                    search_term = """//*[contains(@title, 'change')]"""
                    search_term = search_term.replace("change", tag.text)
                else:    
                    search_term = """//*[contains(@title, "change")]"""
                    search_term = search_term.replace("change", tag.text)
                
                # Locate card and click
                item = driver.find_element_by_xpath(search_term)
                item.click()
                
                # Trying to locate the right info box from google, since it can of different type so handling both cases
                try:
                    
                    input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': "kp-wp-tab-overview"}

                    item = find_element(driver, input_dict)
                    item = item.get_attribute('innerHTML')
                    soup = BeautifulSoup(item, "html.parser")
                    
                    # Getting the description element
                    html_content = str(soup.find("div", {"class": "kno-rdesc"}))
                    
                except:
                    input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': 'rhs'}

                    item = find_element(driver, input_dict)

                    item = item.get_attribute('innerHTML')

                    soup = BeautifulSoup(item, "html.parser")

                    html_content = str(soup.find("div", {"class": "kno-rdesc"}))
                
                # Get the description
                description = html_content[html_content.find("<span>") + len("<span>"):html_content.find("</span>")]

                output_dict[tag.text] = description.replace("\n","")
        
        return output_dict
    
    except:
        # Getting the cards of gov agencies if the format is different
        input_dict = {'search_by': 'by_class','find_type': 'single', 'find_str': "rl_container"}

        item = find_element(driver, input_dict)
        item = item.get_attribute('innerHTML')
        soup = BeautifulSoup(item, "html.parser")
        
        # Traversing through all gov agency cards 
        for tag in soup.find_all("a", href=True):
            if tag.text:
                temp = driver.find_element_by_link_text(tag.text)
                temp.click()
                try:
                    input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': "kp-wp-tab-overview"}

                    item = find_element(driver, input_dict)

                    item = item.get_attribute('innerHTML')
                    soup = BeautifulSoup(item, "html.parser")
                    html_content = str(soup.find("div", {"class": "kno-rdesc"}))
                except:
                    input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': 'rhs'}

                    item = find_element(driver, input_dict)

                    item = item.get_attribute('innerHTML')

                    soup = BeautifulSoup(item, "html.parser")

                    html_content = str(soup.find("div", {"class": "kno-rdesc"}))
                
                description = html_content[html_content.find("<span>") + len("<span>"):html_content.find("</span>")]

                output_dict[tag.text] = description.replace("\n","")
        
        return output_dict
        
########################################################## 
# Using the googlesearch python package to get urls
def get_link_google(search_term, pages, number):
    output = []
    item = search(search_term, tld="co.in", num=number, stop=pages, pause=2)
    for j in item: 
        output.append(j)
    return output

########################################################## 
# To get the infobox from wikipedia
def get_infobox(search_term):
    
    driver = open_browser(driver_path)
    driver.get('https://en.wikipedia.org/wiki/Wiki')
    time.sleep(random.randint(1,2))
    
    # Get the search bar in wiki
    input_dict = {'search_by': 'by_id','find_type': 'single', 'find_str': 'searchInput'}

    # Getting search bar and click on it
    body = find_element(driver, input_dict)
    body.click()
    
    # Enter the search term in the search bar
    pyautogui.write(search_term)
    time.sleep(1)
    
    # Using pyautogui to press down key to make sure we click on the best suggestion by wikipedia.
    # This ensures that if we enter a search term which is not right, wiki might correct it and suggest the best match
    pyautogui.press('down')
    time.sleep(1)
    pyautogui.press('enter')

    time.sleep(random.randint(1,2))
    link = driver.current_url
    
    # Getting the html of infobox in right 
    infoboxes = read_html(link, index_col=0,  attrs={"class":"infobox"})
    
    # Parsing the required information from infobox
    infobox = infoboxes[0].to_dict()

    infobox = list(infobox.values())

    infobox = infobox[0]
    driver.quit()
    
    return infobox
    
########################################################## 
# To convert keys of dict to lowercase
def convert_key_lower(dict_):
    try:
        return dict((k.lower(), v) for k, v in dict_.items()) 
    except:
        return dict_

## Creating structure for final output

In [17]:
result_df = pd.DataFrame(columns = ["Country", "GA_Name", "Address","Contact","Off_website", "Description"])
result_df

,Country,GA_Name,Address,Contact,Off_website,Description


## Get all country name in the world

### If the country names are already extracted and saved, reading from the file

In [23]:
country_df = pd.read_csv( str(os.getcwd()) + "/Data/country_df.csv")

### Getting all country name from url to get country_df

In [ ]:
driver = open_browser(driver_path)

# Feeding url
driver.get('https://www.worldometers.info/geography/alphabetical-list-of-countries/')

# Creating input dict for find_element() 
input_dict = {'search_by': 'by_class','find_type': 'single', 'find_str': 'table-responsive'}

# Getting table element
table = find_element(driver, input_dict)

# Getting the html code of table to parse all columns
table = table.get_attribute('innerHTML')

parser = etree.HTMLParser()
tree = etree.fromstring(table, parser)

# Parsing columns 2,3,4,5 in the webpage table
country_names = tree.xpath('//tr/td[position()=2]')
Population_2020 = tree.xpath('//tr/td[position()=3]/a')
Land_Area_Km2 = tree.xpath('//tr/td[position()=4]')
Density_P_Km2 = tree.xpath('//tr/td[position()=5]')

# Re-creating the table
country_dict = {}
for i in range(0,len(country_names)):
    country_dict[country_names[i].text] = {"Population_2020": int(Population_2020[i].text.replace(",","")), 
                                              "Land Area_Km²": int(Land_Area_Km2[i].text.replace(",","")), 
                                              "Density_P/Km²": int(Density_P_Km2[i].text.replace(",",""))}

country_df = pd.DataFrame.from_dict(country_dict, orient='index')

country_df.reset_index(inplace = True)

country_df = country_df.rename(columns={'index': 'Country'})

country_df.to_csv(str(os.getcwd()) + "/Data/country_df.csv")

# Checking if we got 195 countries
print("There should be 195 countries: ")
print(country_df.shape())

country_df.head(2)



## Getting government agencies country wise

### Getting all gov agency for each country from Google

In [ ]:
# Creating a list to capture the rows where we could not get the gov agencies to analyse it later
error_items = []

# Iterating through countries
for Country in country_df['Country']:
    try:
        output = get_gov_agency(Country)
    #     print(output)
        for key in output.keys():
            GA_Name = key
            Description = output[key]
            df = pd.DataFrame({"Country": [Country], 
                                "GA_Name":[GA_Name],  
                                "Address":[""],
                                "Contact": [""],
                                "Off_website": [""],
                                "Description": [Description]}) 
            result_df = result_df.append(df, ignore_index = True)
    except:
        error_items.append(Country)
        continue
    driver.quit()

# Removing extra whitespaces from Gov Agency names
for column in result_df.columns:
    try:
        result_df[column] = result_df[column].apply(lambda x: x.strip())
    except:
        print("Error with column:" + column)
        continue

In [ ]:
result_df.to_csv(str(os.getcwd()) + "/Data/result_df.csv")

## Checking countries for which we could not scrape Government agencies names

In [236]:
country_left = []
for i in list(country_df['Country'].unique()):
    if i not in list(result_df['Country'].unique()):
        country_left.append(i)
country_left

['Angola',
 'Antigua and Barbuda',
 'Benin',
 'Burkina Faso',
 'Burundi',
 "Côte d'Ivoire",
 'Cabo Verde',
 'Cameroon',
 'Central African Republic',
 'Chad',
 'Comoros',
 'Congo (Congo-Brazzaville)',
 'Democratic Republic of the Congo',
 'Djibouti',
 'Dominica',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Eswatini (fmr. "Swaziland")',
 'Gabon',
 'Gambia',
 'Grenada',
 'Guinea',
 'Guinea-Bissau',
 'Honduras',
 'Kiribati',
 'Liechtenstein',
 'Madagascar',
 'Mali',
 'Marshall Islands',
 'Mauritania',
 'Micronesia',
 'Monaco',
 'Mozambique',
 'Myanmar (formerly Burma)',
 'Nauru',
 'Niger',
 'Palau',
 'Palestine State',
 'Paraguay',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Samoa',
 'Senegal',
 'Seychelles',
 'Solomon Islands',
 'Suriname',
 'Tajikistan',
 'Togo',
 'Tonga',
 'Tuvalu',
 'Vanuatu',
 'Yemen']

## Getting information of Government agencies from infobox from Wikipedia

In [166]:
# To improve accuracy of input we gather, I tried to keep country name with the Gov Agency name so that we get most accurate result

# If we are not able to extract the infobox we store the row number in error_rows to analyse later

error_rows = []
for i in range(0, len(result_df)):
    try:
        try:
            country = result_df['Country'][i].lower()
            gov_ag_Name = result_df['GA_Name'][i].lower()
            
            # Scenario 1: Checking if Gov Agency Name is like "Gov Agency of Country"
            if ("of " + country) in gov_ag_Name: 
                try:
                    input_ = gov_ag_Name
                    infobox = get_infobox(input_)   
                    
                # If we are not able to get the infobox we change the string "Gov Agency of Country" to "Gov Agency (Country)"    
                except:
                    pyautogui.hotkey('ctrl', 'w')
                    input_ = gov_ag_Name.replace("of "+country, "(" + country) + ")"                
                    infobox = get_infobox(input_)
            
            # Scenario 2: Checking if Country name present in Gov Agency Name
            elif country in gov_ag_Name:
                
                # Checking if "(Country) present in Gov Afency Name" and trying to get infobox
                if "(" + country + ")" in gov_ag_Name:
                    input_ = gov_ag_Name
                    infobox = get_infobox(input_)
                
                # If cannot get infobox then change "Country Name" to "(Country Name)"
                else:
                    input_ = gov_ag_Name
                    input_ = input_.replace(country, "(" + country + ")" )
                    infobox = get_infobox(input_)
            
            # Scenario 3: If Country Name not present in Gov Agency Name, adding the country name to increase accuracy
            else:
                input_ = gov_ag_Name + " (" + country + ")"
                infobox = get_infobox(input_)
                
        # Scenario 4: Feeding the Gov Agency Name with no change        
        except: 
            pyautogui.hotkey('ctrl', 'w')
            input_ = gov_ag_Name
            infobox = get_infobox(input_)

        result_df['wiki_infobox'][i] = infobox
    except Exception as e:
        error_rows.append(i)
        continue
        
result_df.to_csv(str(os.getcwd()) + "/Data/result_df.csv")

/home/sumit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Getting the different types of fields from the infobox for different gov agencies to fill columns in result_df

In [40]:
key_list_unique = []
key_list = []
for i in range(0,len(result_df)):
    try:
        dict_ = result_df['wiki_infobox'].loc[i]
        
        if type(dict_) == str:
            dict_ = ast.literal_eval(result_df['wiki_infobox'].loc[i])
            
        for key,value in dict_.items():
            if key not in key_list_unique:
                key_list_unique.append(key)

            key_list.append(key)
    except:
        continue

In [43]:
print(len(key_list))
print(len(key_list_unique))

11685
2076


### Taking the count of extracted field names to assess the most common fields

In [33]:
key_count={}
for key in key_list:
    try:
        value = key_count[key]
        key_count[key] = value+1
    except:
        key_count[key] = 1

### Sorting the fields by count of keys to identify and use the important columns

In [34]:
{k: key_count[k] for k in sorted(key_count, key=key_count.get, reverse=True)}

{'website': 1141,
 'headquarters': 1122,
 'formed': 883,
 'jurisdiction': 836,
 'agency overview': 744,
 'agency executive': 462,
 'minister responsible': 369,
 'parent agency': 261,
 'annual budget': 254,
 'employees': 250,
 'agency executives': 177,
 'preceding agency': 131,
 'type': 128,
 'operational structure': 98,
 'jurisdictional structure': 97,
 'operations jurisdiction': 95,
 'ministry overview': 86,
 'abbreviation': 83,
 'parent department': 83,
 'preceding agencies': 82,
 'dissolved': 70,
 'general nature': 64,
 'ministers responsible': 62,
 'child agencies': 61,
 'motto': 59,
 'superseding agency': 54,
 'formation': 54,
 'child agency': 50,
 'established': 47,
 'ministry executive': 42,
 'national agency': 36,
 'governing body': 36,
 'ownership': 36,
 'central\xa0bank\xa0of': 36,
 'currency': 34,
 'reserves': 34,
 'deputy minister responsible': 33,
 'department overview': 32,
 'department executive': 30,
 'country': 29,
 'elected officer responsible': 28,
 'governor': 27,
 

### Getting all probable field names which can give the official website from the infobbox information

In [208]:
probable_website_keys = []
for i in key_list_unique:
    if "website" in str(i).lower():
        probable_website_keys.append(str(i).lower())

### Getting all probable field names which can give the address from the infobbox information

In [220]:
probable_headquarter_keys = []
for i in key_list_unique:
    if "headquarter" in str(i).lower() or "head office" in str(i).lower() or "office " in str(i).lower() or "address" in str(i).lower():
        probable_headquarter_keys.append(str(i).lower())

### Getting all probable field names which can give the formed date from the infobbox information

In [210]:
probable_formed_keys = []
for i in key_list_unique:
    if "formed" in str(i).lower() or 'founded' in str(i).lower():
        if i != "Founded by":
            probable_formed_keys.append(str(i).lower())

### Getting all probable field names which can give the jurisdiction from the infobbox information

## Creating new columns to store the values from infobox

In [37]:
result_df['Formed'] = ""
result_df['Jurisdiction'] = ""
result_df.columns


Index(['Country', 'GA_Name', 'Formed', 'Description', 'Off_website', 'Address',
       'Jurisdiction', 'wiki_infobox'],
      dtype='object')

In [38]:
# Converting the keys of infobox in each row to lowercase to increase chances of match later
result_df['wiki_infobox'] = result_df['wiki_infobox'].apply(lambda x: convert_key_lower(x))

In [54]:
# Extracting official website, address, formed date and jurisdiction from infobox with best possible match
for i in range(0,len(result_df)):
    infobox_details = result_df['wiki_infobox'][i]
    
    if type(infobox_details) == str:
        try:
            infobox_details = ast.literal_eval(infobox_details)
        except:
            pass
    
    for web_key in probable_website_keys:
        try:
            result_df['Off_website'][i] = infobox_details[web_key]
            break
        except:
            continue
    
    for hq_key in probable_headquarter_keys:
        try:
            result_df['Address'][i] = infobox_details[hq_key]
            break
        except:
            continue
            
    for form_key in probable_formed_keys:
        try:
            result_df['Formed'][i] = infobox_details[form_key]
            break
        except:
            continue
            
    for jur_key in probable_jurisdiction_keys:
        try:
            result_df['Jurisdiction'][i] = infobox_details[jur_key]
            print(infobox_details[jur_key])
            break
        except:
            continue

Afghanistan
Government of Afghanistan
Government of Afghanistan
Government of Afghanistan
Government of India
Jurisdictional structure
Government of Afghanistan
Government of Afghanistan
Government of India
Government of Afghanistan
Jurisdictional structure
Albania
Albania
Jurisdictional structure
Government of Albania
Albania
Albania
Albania
Albania
Jurisdictional structure
Albania
Albania
Albania
Albania
Albania
Albania
Albania
Albania
Albania
Albania
Albania
Jurisdictional structure
Jurisdictional structure
Albania
Albanian Government
Albania
Albania
Government of Algeria
Democratic People 's Republic of Algeria
Jordan
Jurisdictional structure
Jurisdictional structure
Government of Armenia
Government of Armenia
Armenia
Jurisdictional structure
Jurisdictional structure
Government of Bahrain
Government of Bahrain
Cabinet of Bahrain
Government of Bahrain
Government of Bahrain
Cabinet of Bahrain
Government of Bahrain
Government of Belarus
Belarus
Council of Ministers of Belarus
Armed Fo

Jurisdictional structure
Republic of China (Taiwan)
Government of New Zealand
New Zealand
New Zealand
Jurisdictional structure
New Zealand
New Zealand
New Zealand
New Zealand
Federal Government of Pakistan
Government of Pakistan
Pakistan
Government of Pakistan
Pakistan
Government of Pakistan
Government of Pakistan
Constitution of Pakistan
Government of Pakistan
Government of Pakistan
Jurisdictional structure
Pakistan
Philippines
Jurisdictional structure
Philippines
Government of the Philippines
National
Government of the Philippines
Jurisdictional structure
Jurisdictional structure
Philippines
Poland
Jurisdictional structure
President of Russia
President of Russia
Jurisdictional structure
President of Russia
Government of Russia
Government of Russia
Government of Russia
Jurisdictional structure
Government of Russia
Government of Russia
Government of Russia
Government of Russia
Government of Russia
Ministry of Transport
Prime Minister of Russia
Saudi Arabia
Government of Saudi Arabia
Sa

In [46]:
# Rearranging the columns
result_df = result_df[['Country', 'GA_Name', 'Formed', 'Description', 'Off_website', 'Address', 'Jurisdiction',  'wiki_infobox']]
result_df.columns

Index(['Country', 'GA_Name', 'Formed', 'Description', 'Off_website', 'Address',
       'Jurisdiction', 'wiki_infobox'],
      dtype='object')

## Dropping entries for China as asked in the problem statement

In [47]:
final_df = result_df.loc[result_df['Country'] != "China"]
final_df.reset_index(inplace=True, drop=True)
print(result_df.shape)
print(final_df.shape)

final_df.to_csv(str(os.getcwd()) + "/Data/final_df.csv")


(4129, 8)
(4129, 8)


## Rough